In [2]:
!pip install xpander-sdk openai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from xpander_sdk import XpanderClient, LLMProvider
from openai import OpenAI
import json

## Load environment variables
from dotenv import load_dotenv
import os
load_dotenv()
xpanderAPIKey = os.environ.get("XPANDER_API_KEY","") ## Your XPANDER API Personal Key
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "") ## Your Agent ID
openAIKey = os.environ.get("OPENAI_API_KEY", "") ## Your Agent ID

openai_client = OpenAI(api_key=openAIKey) ## Your OpenAI API Key

## Connecting to the agent
xpander_client = XpanderClient(api_key=xpanderAPIKey)
agent1 = xpander_client.agents.get(agent_id=xpanderAgentID)

agent1.get_tools()

[{'type': 'function',
  'function': {'name': 'PgCrunchbaseLinkedInPerplexityDataEnrichment',
   'description': "This operation enriches company data by integrating information from LinkedIn, Crunchbase, and Perplexity. Use it for comprehensive company analysis, retrieving employee details, or gathering the latest news and trends related to a specific company like Nvidia. It's particularly useful for market researchers, analysts, and business developers looking to enhance their understanding of a company's profile. IMPORTANT! Ensure to use body_params, query_params, path_params. These are crucial for correct function calling!",
   'parameters': {'type': 'object',
    'properties': {'queryParams': {'type': 'object',
      'properties': {},
      'required': []},
     'pathParams': {'type': 'object', 'properties': {}, 'required': []},
     'bodyParams': {'type': 'object',
      'properties': {'inputTask': {'type': 'string',
        'description': 'input user sub task based on the input qu

In [5]:
memory = []
memory.append({"role": "system", "content": "You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## together with the final answer"})
print(memory[-1])
memory.append({"role": "user", "content": "Get news about qwen2.5-coder"})
print(memory[-1])
number_of_calls = 1

while True:
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    print(memory[-1])
    llm_response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=memory,
        tools=agent1.get_tools()    
    )
    memory.append(llm_response.choices[0].message)
    print(memory[-1])
    if(llm_response.choices[0].message.tool_calls):
        print(llm_response.choices[0].message.tool_calls)
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump())
        tool_response = agent1.run_tools(tools_to_run)
        for tool_response in tool_response:
            tool_result = {
                "Selected_Function_name": tool_response.function_name,
                "Http_Error_Code": tool_response.status_code,
                "Response_from_Target_System": json.dumps(tool_response.result),
                "Generated_Payload_From_AI": tool_response.payload,
                "Original_Tool_Id": tool_response.tool_call_id
            }
            print(tool_result)
            memory.append({"role": "tool", "content": json.dumps(tool_response.result), "tool_call_id": tool_response.tool_call_id})
    if(llm_response.choices[0].message.content):
        if "##FINAL ANSWER##" in llm_response.choices[0].message.content:
            break
    number_of_calls += 1
print(llm_response.choices[0].message.content)

{'role': 'system', 'content': 'You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## together with the final answer'}
{'role': 'user', 'content': 'Get news about qwen2.5-coder'}
{'role': 'assistant', 'content': 'Step number: 1'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ObZEaBXR0D409ayEMiPhfbbp', function=Function(arguments='{"bodyParams":{"query":"qwen2.5-coder"},"queryParams":{},"pathParams":{}}', name='NotionContentSearchSearchPagesAndDatabases'), type='function')])
[ChatCompletionMessageToolCall(id='call_ObZEaBXR0D409ayEMiPhfbbp', function=Function(arguments='{"bodyParams":{"query":"qwen2.5-coder"},"queryParams":{},"pathParams":{}}', name='NotionContentSearchSearchPagesAndDatabases'), type='function')]
{'Selected_Function_name': 'NotionContentSearchSearchPagesAndDatabases', 'H

# Return all tools

In [27]:
agent1.get_tools(return_all_tools=True)

[{'type': 'function',
  'function': {'name': 'PgCrunchbaseLinkedInPerplexityDataEnrichment',
   'description': "This operation enriches company data by integrating information from LinkedIn, Crunchbase, and Perplexity. Use it for comprehensive company analysis, retrieving employee details, or gathering the latest news and trends related to a specific company like Nvidia. It's particularly useful for market researchers, analysts, and business developers looking to enhance their understanding of a company's profile. IMPORTANT! Ensure to use body_params, query_params, path_params. These are crucial for correct function calling!",
   'parameters': {'type': 'object',
    'properties': {'queryParams': {'type': 'object',
      'properties': {},
      'required': []},
     'pathParams': {'type': 'object', 'properties': {}, 'required': []},
     'bodyParams': {'type': 'object',
      'properties': {'inputTask': {'type': 'string',
        'description': 'input user sub task based on the input qu

# Local tools

In [33]:
from xpander_sdk import ToolCall, ToolCallType

toolcall = ToolCall(
    name="tavily-insights-fetchInsightsFromTavilyAI",
    type=ToolCallType.XPANDER,
    payload={
        "bodyParams": {
            "query": "qwen2.5-coder news on HackerNews",
            "max_results": "5",
            "include_raw_content": "true",
            "include_images": "false",
            "exclude_domains": ["example.com"], 
        },
        "queryParams": {},
        "pathParams": {}
    }
)
result = agent1.run_tool(toolcall)
print(result.result)

{'query': 'qwen2.5-coder news on HackerNews', 'images': [], 'results': [{'title': "Qwen2.5-Coder just changed the game for AI programming—and it's free", 'url': 'https://venturebeat.com/ai/alibaba-new-ai-can-code-in-92-languages-and-its-completely-free/', 'content': 'Alibaba Cloud has released Qwen2.5-Coder, a new AI coding assistant that has already become the second most popular demo on Hugging Face Spaces. The release includes six model variants, from 0.5 billion to 32 billion parameters, making advanced AI coding accessible to developers with different computing resources. A comparison of AI coding models shows Alibaba’s Qwen2.5-Coder-32B (in blue) outperforming GPT-4 and other competitors across multiple industry benchmarks. Benchmark results comparing Alibaba’s Qwen2.5-Coder against leading AI models, including GPT-4 and Claude 3.5. The model’s release intensifies the AI development race between the U.S. and China. For developers and businesses worldwide, Qwen2.5-Coder presents a